In [19]:
from lrfhss.run import *
from lrfhss.traffic import *
import time
from joblib import Parallel, delayed
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pickle
import pandas as pd
from matplotlib.ticker import EngFormatter
import scienceplots

# Example 1: Success Probability versus Number of Nodes

Here is a simple example on how to run the simulator using most default input parameter from the class Settings. This is defined in the file runSim, which uses from the simulator classes to create the simulation environment. You can check the file to modified or create you own modified version.

We run the simulation for several different number of devices, and also run it 5 times and get the average and standard deviation of the results. The data is stored in a pandas Dataframe and saved on a .data file.

In [ ]:
s = Settings(sensitivity=140)
print(s.sensitivity)
run_sim(s)


AttributeError: 'Settings' object has no attribute 'sensitivity'